In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_selection import RFE
rf = RandomForestClassifier()
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from scipy.stats import randint

In [2]:
data = pd.read_csv('~/Desktop/project_data_new/embedding_768_TCGA_COAD.csv')
data.index = data['PatientID']
# drop last 2 columns
data = data.drop(data.columns[-2:], axis=1)

In [3]:
data_target = pd.read_csv('~/Desktop/project_data_new/target_768_avg_expanded.csv')

In [4]:
data_target.index = data_target['Unnamed: 0']
data_target = data_target.drop(['Unnamed: 0'], axis = 1)
# only keep the columns with category in the name
data_target = data_target.loc[:, data_target.columns.str.contains('category')]

In [5]:
data = data[data.index.isin(data_target.index)]

In [6]:
data_target

,category_tnk_1,category_tnk_2,category_tnk_3,category_tnk_4,category_tnk_5,category_tnk_6,category_tnk_7,category_tnk_8,category_tnk_9,category_tnk_10,...,category_stromal_62,category_stromal_63,category_stromal_64,category_stromal_65,category_stromal_66,category_stromal_68,category_stromal_73,category_stromal_78,category_stromal_80,category_stromal_85
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
TCGA-A6-6650,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,1,1
TCGA-A6-6142,1,1,0,0,0,0,0,0,-1,0,...,1,1,1,1,1,-1,0,0,-1,-1
TCGA-5M-AAT4,-1,-1,-1,0,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,1,1
TCGA-A6-2676,1,1,1,0,1,1,1,1,1,1,...,0,-1,0,-1,0,1,0,1,1,1
TCGA-A6-6652,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-G4-6321,1,1,1,-1,1,1,1,1,-1,1,...,0,0,-1,1,1,-1,1,1,-1,-1
TCGA-RU-A8FL,-1,-1,-1,1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,1,1
TCGA-G4-6303,-1,0,-1,-1,0,0,0,-1,-1,1,...,1,1,1,0,1,-1,0,0,-1,-1


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
from scipy.stats import randint

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Extract the feature values from data
X = data.values

# Set the chunk size (number of target variables per part)
chunk_size = 40

# Loop through target columns in chunks
for chunk_start in range(0, 241, chunk_size):
    # Get the current chunk of target columns
    chunk_end = min(chunk_start + chunk_size, 241)
    target_chunk = data_target.columns[chunk_start:chunk_end]
    
    # Initialize list to store results for the current chunk
    chunk_results = []

    # Open a file to write classification reports for the current chunk
    report_file_path = f"/home/qiuaodon/Desktop/CRC_image/Best_features_REF/Classification_Reports_100features_part_{chunk_start}_{chunk_end}.txt"
    with open(report_file_path, "w") as report_file:
        
        for target_col in target_chunk:
            # Extract the target column for the current category
            Y = data_target[target_col].values

            # Select top 100 features using RFE
            rfe = RFE(rf, n_features_to_select=100)  # Choose top 100 features
            X_selected = rfe.fit_transform(X, Y)
            
            # Split the data into training and testing sets
            X_train, X_test, Y_train, Y_test = train_test_split(X_selected, Y, test_size=0.2, random_state=42)

            # Define the hyperparameter distribution
            param_dist = {
                'n_estimators': randint(100, 200),
                'max_depth': randint(20, 40),
                'min_samples_split': randint(10, 20),
                'min_samples_leaf': randint(1, 5),
                'max_features': ['sqrt', 'log2']
            }

            # Perform Randomized Search with cross-validation
            random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=60, 
                                               cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
            random_search.fit(X_train, Y_train)

            # Get the best model from Randomized Search
            best_rf = random_search.best_estimator_
            best_params = random_search.best_params_

            # Fit the model on the training data for evaluation
            best_rf.fit(X_train, Y_train)

            # Evaluate on the training set
            Y_train_pred = best_rf.predict(X_train)
            train_report = classification_report(Y_train, Y_train_pred)
            
            # Evaluate on the test set
            Y_test_pred = best_rf.predict(X_test)
            test_report = classification_report(Y_test, Y_test_pred, output_dict=True)

            # Write classification reports to the single file
            report_file.write(f"Classification Report for {target_col} (Train Set):\n")
            report_file.write(train_report)
            report_file.write("\n")
            report_file.write(f"Classification Report for {target_col} (Test Set):\n")
            report_file.write(classification_report(Y_test, Y_test_pred))
            report_file.write("\n\n" + "="*80 + "\n\n")

            # Calculate and store precision, recall, and accuracy
            precision = precision_score(Y_test, Y_test_pred, average='weighted')
            recall = recall_score(Y_test, Y_test_pred, average='weighted')
            accuracy = accuracy_score(Y_test, Y_test_pred)

            # Extract precision, recall, and f1-score specifically for class '1'
            class_1_metrics = test_report.get('1', {"precision": None, "recall": None, "f1-score": None})

            chunk_results.append({
                "Target Variable": target_col,
                "Precision": precision,
                "Recall": recall,
                "Accuracy": accuracy,
                "Class 1 Precision": class_1_metrics["precision"],
                "Class 1 Recall": class_1_metrics["recall"],
                "Class 1 F1-Score": class_1_metrics["f1-score"],
                "Best Hyperparameters": best_params
            })
    
    # Save the results for the current chunk to an Excel file
    results_df = pd.DataFrame(chunk_results)
    results_df.to_excel(f"/home/qiuaodon/Desktop/CRC_image/Best_features_REF/Precision_Recall_Accuracy_100features_part_{chunk_start}_{chunk_end}.xlsx", index=False)


Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits


In [9]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
from scipy.stats import randint

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Extract the feature values from data
X = data.values

# Initialize lists to store precision, recall, and accuracy for each target variable
results = []

# Open a single file to write all classification reports
with open("/home/qiuaodon/Desktop/CRC_image/Best_features_REF/Classification_Reports_100features.txt", "w") as report_file:
    
    # Loop through the first 120 target columns in data_target
    for i, target_col in enumerate(data_target.columns[:241]):
        # Extract the target column for the current category
        Y = data_target[target_col].values

        # Select top 100 features using RFE
        rfe = RFE(rf, n_features_to_select=100)  # Choose top 100 features
        X_selected = rfe.fit_transform(X, Y)
        
        # Split the data into training and testing sets
        X_train, X_test, Y_train, Y_test = train_test_split(X_selected, Y, test_size=0.2, random_state=42)

        # Define the hyperparameter distribution (use distributions for random search)
        param_dist = {
            'n_estimators': randint(100, 200),
            'max_depth': randint(20, 40),
            'min_samples_split': randint(10, 20),
            'min_samples_leaf': randint(1, 5),
            'max_features': ['sqrt', 'log2']
        }

        # Perform Randomized Search with cross-validation
        random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=60, 
                                           cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
        random_search.fit(X_train, Y_train)

        # Get the best model from Randomized Search
        best_rf = random_search.best_estimator_
        best_params = random_search.best_params_

        # Fit the model on the training data for the final evaluation on train and test sets
        best_rf.fit(X_train, Y_train)

        # Evaluate on the training set
        Y_train_pred = best_rf.predict(X_train)
        train_report = classification_report(Y_train, Y_train_pred)
        
        # Evaluate on the test set
        Y_test_pred = best_rf.predict(X_test)
        test_report = classification_report(Y_test, Y_test_pred, output_dict=True)

        # Write classification reports to the single file
        report_file.write(f"Classification Report for {target_col} (Train Set):\n")
        report_file.write(train_report)
        report_file.write("\n")
        report_file.write(f"Classification Report for {target_col} (Test Set):\n")
        report_file.write(classification_report(Y_test, Y_test_pred))
        report_file.write("\n\n" + "="*80 + "\n\n")

        # Calculate and store precision, recall, and accuracy
        precision = precision_score(Y_test, Y_test_pred, average='weighted')
        recall = recall_score(Y_test, Y_test_pred, average='weighted')
        accuracy = accuracy_score(Y_test, Y_test_pred)

        # Extract precision, recall, and f1-score specifically for class '1'
        class_1_metrics = test_report.get('1', {"precision": None, "recall": None, "f1-score": None})

        results.append({
            "Target Variable": target_col,
            "Precision": precision,
            "Recall": recall,
            "Accuracy": accuracy,
            "Class 1 Precision": class_1_metrics["precision"],
            "Class 1 Recall": class_1_metrics["recall"],
            "Class 1 F1-Score": class_1_metrics["f1-score"],
            "Best Hyperparameters": best_params
        })

# Save the results to an Excel file
results_df = pd.DataFrame(results)
results_df.to_excel("/home/qiuaodon/Desktop/CRC_image/Best_features_REF/Precision_Recall_Accuracy_100features.xlsx", index=False)


Fitting 5 folds for each of 60 candidates, totalling 300 fits


KeyboardInterrupt: 